## student scrapper activity
scrape categories and all books for each category and then insert into mongo db

In [65]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import pdb
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,TimeoutException
import time
import pandas as pd
import numpy as np

In [66]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/davidcoy/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [67]:
def author_data(i):
        browser.find_by_text('(about)')[i].click()
        html = browser.html
        soup_about = BeautifulSoup(html,'html.parser')
        date_born = soup_about.find_all('span', class_='author-born-date')[0].text
        location = soup_about.find_all('span', class_='author-born-location')[0].text
        born = f'{date_born} {location}'
        description = soup_about.find_all('div', class_='author-description')[0].text
        browser.back()
        time.sleep(0.25)
        return(born,description)

In [68]:
url = 'http://quotes.toscrape.com/'
#url = 'http://quotes.toscrape.com/page/10/'
browser.visit(url)
quotes_list = []

#if browser.find_by_text('Next ').first:

for x in range(10):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html,'html.parser')
    quotes = soup.find_all('div', class_='quote')
    i=0
    for quote in quotes:
        tag_list = []
        tags = quote.find_all('a', class_='tag')
        for tag in tags:
            tag_list.append(tag.text)
        time.sleep(0.5)
        born,description = author_data(i)

        #pdb.set_trace()
        data = {'quote' : quote.span.text,
                'author' : quote.small.text,
                'tags' : tag_list,
                'born': born,
                'description': description
               }
        quotes_list.append(data)
        i=i+1
    time.sleep(0.25)
    try:    
        print('Clicking!')
        browser.find_by_text('Next ').first.click()
        time.sleep(0.25)
    except:
        print('Done Scraping')

quotes_list
browser.quit()

Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Clicking!
Done Scraping


In [64]:
browser.quit()

In [69]:
pd.DataFrame(quotes_list)

,quote,author,tags,born,description
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]","July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]","March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]","December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]","June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy],"April 28, 1926 in Monroeville, Alabama, The Un...","\n Harper Lee, known as Nelle, was born..."
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...","November 29, 1918 in New York City, New York, ...",\n Madeleine L'Engle was an American wr...
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth],"November 30, 1835 in Florida, Missouri, The Un...","\n Samuel Langhorne Clemens, better kno..."
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational],"March 02, 1904 in Springfield, MA, The United ...",\n Theodor Seuss Geisel was born 2 Marc...
